# Scraping the Broadcast News of Russian Channel One Web Site

In [2]:
import os.path
import csv
import scrapy
import requests
import pandas as pd
import dateparser
from scrapy.crawler import CrawlerProcess
from scrapy.exporters import CsvItemExporter

# dateparser is used to parse date in different languages other than English (Russian in this case)
# https://stackoverflow.com/questions/41051389/pandas-parse-non-english-string-dates

# See the required dependencies at the end of this notebook


In [12]:
# Dependencies
# %pip install watermark
# %load_ext watermark
# %watermark -iv

requests  : 2.28.1
scrapy    : 2.7.1
csv       : 1.0
sys       : 3.10.7 (tags/v3.10.7:6cc6b13, Sep  5 2022, 14:08:36) [MSC v.1933 64 bit (AMD64)]
dateparser: 1.1.6
pandas    : 1.5.0



Before digging into the actual scraping, let's see an example of connecting to the web site of *Channel One* and extract broadcast articles titles using a `Selector` object from the `scrapy` library using the `css` notation to access the `html` hierarchy (`xpath` notation could also be used).

In [3]:
# url = "https://www.1tv.ru/news"
# html = requests.get(url).content
# sel = scrapy.Selector(text=html)
# sel.css("article > a > div > div.title > h2::text").extract()


In [4]:
# url = "https://www.1tv.ru/news/2022-12-22/444010-vladimir_putin_ob_atakah_zapada_na_russkiy_mir_i_nashem_otvete"
# html = requests.get(url).content
# sel = scrapy.Selector(text=html)
# body = sel.xpath(
#            "//div[@class = 'editor text-block active' or @class = 'editor text-block']//text()"
#        ).extract()
# body

Some definitions before starting the explanation of the scraping process
* the **offset** specifies where to start a page

* the **limit** specifies the page size in the query about the request to the server

By looking at the Channel One News page, it is possible to recognize that it is a *partial infinite scrolling page*. At the time of writing, the web page shows the first 15 news (`limit = 15`, `offset = 0`; see the definition below) and the user should click *show more* to see more news (`limit = 15`, `offset = 15`)

![](https://drive.google.com/uc?export=view&id=17n5brqhux7GqAp9_n_F82Gx34Ze5YRyJ) 

After that the page scrolling infinitely as the user continues scrolling. In other words the browser requests the server to show additional 15 more entries (`offset = offset + 15`, `limit = 15`). The Occasionaly, the user is asked to click *show more* again. 

The Spider replicates this behavior sending requests to the server of showing 15 more additional entries 

The second part of the Spider is to enter inside each news article and scrape the data we are interested in (e.g. date, article title, article body, tags)

In [ ]:
class Propaganda(scrapy.Spider):

    name = "propaganda"  # give whatever name
    allowed_domains = ["1tv.ru"]
    # the initial URL
    LIMIT = 15  # Specifying the page size in the query
    OFFSET = 0 # Change this to set the beginning at a different point in time. OFFSET 0 --> Start from today 21 January 2023
    # OFFSET --> 1500 --> 25 December 2022
    OFFSET_INTERRUPT_AT = 45
    start_urls = ["https://www.1tv.ru/news.js?limit=15&offset=" + str(OFFSET) + "?"]


    def parse(self, response):
        """Three actions
        1. Parsing the front web page 1tv.ru/news by saving locally the html file to scrape in order to collect broadcast urls
        2. For each of these urls the function call the function parse_pages() to scrape each broadcast
        3. Calling itself to scrape other broadcasts in other offsets
        """

        # Step 1. Connecting to the server and saving the html file of each offset locally
        path_folder = os.path.join(".\html_files")
        # e.g. loaded_offset_15.html
        html_file = "loaded_offset_" + str(self.OFFSET) + ".html"
        with open(
            os.path.join(path_folder, html_file), mode="w", encoding="utf-8"
        ) as f1:
            f1.write(response.css("html > body").extract()[0])

        # Opening the file and scraping the list of articles urls
        html_file_to_scrape = "./" + html_file
        with open(
            os.path.join(path_folder, html_file_to_scrape),
            "r",
            encoding="utf-8-sig",
            newline="",
        ) as f2:
            page = f2.read()
            data = scrapy.Selector(text=str(page))
            url_broadcasts = data.css("article>a::attr(href)").extract()
            url_broadcasts = [url.replace("%5C'", "") for url in url_broadcasts]

            # Step 2. For each of these urls the function call the function parse_pages() to scrape each article
            for url in url_broadcasts:
                yield response.follow(url, callback=self.parse_pages)

        # to interrupt the spider, remove this when we are ready
        if self.OFFSET == self.OFFSET_INTERRUPT_AT:
            return None

        # Setting a new offset
        self.OFFSET += 15

        # Step 3. Calling itself to scrape other articles in other offsets
        new_request = (
            "https://www.1tv.ru/news.js?last_date=2023-01-04&limit="
            + str(self.LIMIT)
            + "&offset="
            + str(self.OFFSET)
            + "?"
        )  # the following url to scrape
        yield scrapy.Request(url=new_request, callback=self.parse)

    def parse_pages(self, response):
        """parsing each article page and collecting all the required data"""
        # Broadcast date
        date = response.css("div.date::text").extract()
        date_txt = "".join(date).replace("\xa0", " ")
        # Broadcast tags at the top of the page
        tags_top = response.xpath("//a[@class = 'itv-tag active']/text()").extract()
        # Broadcast title
        title = response.css("h1.title::text").extract()
        title_txt = "".join(title).replace("\xa0", " ")
        # Broadcast description
        body = response.xpath(
            "//div[@class = 'editor text-block active' or @class = 'editor text-block']//text()"
            # 20230128 Issue during testing  "//div[@class = 'editor text-block active']//p//text()"
            # Some news do not have the class 'editor text-block active' but only 'editor text-block';
            # Removed p because some news do not have the paragraph section 
        ).extract()
        body_txt = "".join(body).replace("\xa0", " ")
        # Broadcast tags at the bottom of the page
        tags_bottom = (
            response.xpath(
                "//div[@class='itv-tag-list itv-tag-list--bottom itv-col-7 itv-col-offset-1']"
            )
            .css("a.itv-tag::text")
            .extract()
        )
        # Duration of the video
        video_duration = response.xpath(
            'head/meta[contains(@property,"video:duration")]/@content'
        ).extract()

        yield {
            "date": date_txt,
            "tags_top": tags_top,
            "title": title_txt,
            "url": response.url,
            "body": body_txt,
            "tags_bottom": tags_bottom,
            "video_duration_seconds": video_duration,
        }


# starting the Crawling and define export settings
process = CrawlerProcess(
    settings={
        "FEEDS": {
            "output_scraping.csv": {
                "format": "csv",
                "item_export_kwargs": {
                    "delimiter": "|",
                    "quotechar": '"',
                    "quoting": csv.QUOTE_ALL,
                },
            },
        },
    }
)

process.crawl(Propaganda)
process.start()


In [4]:
df = pd.read_csv(
    "output_scraping.csv",
    delimiter="|",
    date_parser=dateparser.parse,
    parse_dates=["date"],
)
df = df.sort_values(by="date", ascending=False)
df


,date,tags_top,title,url,body,tags_bottom,video_duration_seconds
3,2022-06-03 21:50:00,Общество,Абитуриенты Донецкой и Луганской народных респ...,https://www.1tv.ru/news/2022-06-03/430422-abit...,"Эта новость из Ростова, там проходит первый об...","Общество,Образование,Валерий Фальков",81.0
12,2022-06-03 21:46:00,Технологии,На старте ракеты с Байконура были особые гости...,https://www.1tv.ru/news/2022-06-03/430406-na_s...,Сегодня к Международной космической станции ус...,"Технологии,Космос,Дмитрий Рогозин",276.0
11,2022-06-03 21:45:00,Общество,Уникальные документы об освобождении Николаева...,https://www.1tv.ru/news/2022-06-03/430425-unik...,"Со взятия этого города, превращенного фашистам...","Общество,Образование,Родион Малиновский",78.0
13,2022-06-03 21:44:00,Здоровье,Не стало академика Анатолия Покровского — осно...,https://www.1tv.ru/news/2022-06-03/430408-ne_s...,Ему был 91 год. Покровский один из первых в ми...,"Медицина,Здоровье",50.0
8,2022-06-03 21:43:00,Общество,Большой пожар тушили на западе столицы,https://www.1tv.ru/news/2022-06-03/430421-bols...,Огнем были охвачены четыре этажа бизнес-центра...,"Общество,Пожар,Москва",36.0
...,...,...,...,...,...,...,...
14309,2021-10-17 12:13:00,Технологии,Корреспонденту Первого канала удалось побывать...,https://www.1tv.ru/news/2021-10-17/414862-korr...,Подробнее о самом спускаемом аппарате корабля ...,"Технологии,Космос",123.0
14300,2021-10-17 12:08:00,Технологии,"Интервью с героями киномиссии и теми, благодар...",https://www.1tv.ru/news/2021-10-17/414861-inte...,На месте приземления царит невообразимая суета...,"Технологии,Космос,Кино,Юлия Пересильд,Клим Шип...",262.0
14301,2021-10-17 12:05:00,Технологии,Место встречи киноэкипажа на какое-то время пр...,https://www.1tv.ru/news/2021-10-17/414871-mest...,В расчетной точке в расчетное время. Кружат ве...,"Технологии,Космос,Кино,Юлия Пересильд,Клим Шип...",208.0
14299,2021-10-17 12:00:00,Технологии,Экипаж первой в мире кинокосмической миссии ве...,https://www.1tv.ru/news/2021-10-17/414860-ekip...,Парашют раскрылся на высоте примерно 10 киломе...,"Технологии,Космос,Кино",161.0


In [ ]:
df.head(100)

,date,tags_top,title,url,body,tags_bottom,video_duration_seconds
0,2023-01-24 21:50:00,Общество,Выпуск программы «Время» в 21:00 от 24.01.2023,https://www.1tv.ru/news/2023-01-24/445910-vypu...,Смотрите в этом выпуске: состоялась рабочая вс...,Общество,3193
9,2023-01-24 21:48:00,Культура,Народный артист СССР Юрий Башмет отмечает юбил...,https://www.1tv.ru/news/2023-01-24/445912-naro...,"Альтист-виртуоз, дирижер, руководитель сразу т...","Культура,Музыка,Юрий Башмет",310
10,2023-01-24 21:43:00,Спорт,Международный турнир по быстрым шахматам старт...,https://www.1tv.ru/news/2023-01-24/445907-mezh...,"Сейчас о белых и черных, где нет ничего, кроме...","Спорт,Шахматы,Сергей Корякин,Москва",263
3,2023-01-24 21:42:00,Экономика,Государство продолжит оказывать поддержку моло...,https://www.1tv.ru/news/2023-01-24/445915-gosu...,Россия почти полностью обеспечивает себя молоч...,"Экономика,Сельское хозяйство,Дмитрий Патрушев",73
1,2023-01-24 21:38:00,Экономика,Российская добывающая отрасль активно переходи...,https://www.1tv.ru/news/2023-01-24/445909-ross...,О ситуации с геологоразведкой и разработкой ме...,"Экономика,Геология,Михаил Мишустин,Евгений Петров",250
6,2023-01-24 21:36:00,В мире,Глава МИД РФ Сергей Лавров в рамках африканско...,https://www.1tv.ru/news/2023-01-24/445897-glav...,Западную политику в Африке сегодня комментиров...,"В мире,Международные отношения,Дипломатия,Серг...",101
5,2023-01-24 21:27:00,"Политика,Экономика",Жизненно важные для миллионов россиян вопросы ...,https://www.1tv.ru/news/2023-01-24/445903-zhiz...,"Лекарства (цены и запасы), большой ремонт в ко...","Политика,Экономика,Регионы,Строительство,Медиц...",543
8,2023-01-24 21:23:00,В мире,Германия до конца недели разрешит Варшаве отпр...,https://www.1tv.ru/news/2023-01-24/445908-germ...,Германия до конца недели разрешит Польше перед...,"В мире,Вооружение,Специальная военная операция...",255
4,2023-01-24 21:22:00,В мире,Приостановлены переговоры о вступлении Швеции ...,https://www.1tv.ru/news/2023-01-24/445914-prio...,Натовские устремления Швеции и Финляндии завис...,"В мире,Международные отношения",52
2,2023-01-24 21:16:00,В мире,Экс-глава МИД Польши признал: у Варшавы был пл...,https://www.1tv.ru/news/2023-01-24/445906-eks_...,Теперь европейские скандалы. Их серия сегодня....,"В мире,Международные отношения,Специальная вое...",405


In [5]:
df.to_csv("output_scraping_cleaned.csv", sep="|", quoting=csv.QUOTE_ALL, quotechar='"')
